### Makemore but with MLP

based on this paper: https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf

In [206]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

%matplotlib inline

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

In [207]:
words = open("hindi.txt", "r").read().split()
words = [
    w
    for w in words
    # if "১"
    # and "২"
    # and "৩"
    # and "৪"
    # and "৫"
    # and "৬"
    # and "৭"
    # and "৮"
    # and "৯"
    # and "০" not in w
]

In [ ]:
max(words)

In [209]:
characters = sorted(list((set("".join(words)))))
stoi = {s: i + 1 for i, s in enumerate(characters)}
stoi["~"] = 0
itos = {i: s for s, i in stoi.items()}

In [ ]:
len(itos)

In [ ]:
vocab = len(characters) + 1
vocab

In [ ]:
# building the dataset

block_size = 3  # context length: how many characters do we need to predict the next one
X, Y = [], []
for w in words[:2]:
    print(w)
    context = [0] * block_size
    for ch in w + "~":
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print("".join(itos[i] for i in context), "-->", itos[ix])
        context = context[1:] + [ix]  # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [ ]:
X.shape, X.dtype, Y.shape, Y.dtype

In [214]:
C = torch.randn((vocab, 2))

In [ ]:
F.one_hot(torch.tensor(5), num_classes=vocab).float() @ C

### The Neural Net 🥁

In [216]:
# building the dataset

block_size = 3  # context length: how many characters do we need to predict the next one
X, Y = [], []
for w in words:
    context = [0] * block_size
    for ch in w + "~":
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]  # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)


In [ ]:
X.shape

In [ ]:
C = torch.randn((vocab, 2))
W1 = torch.randn((6, 100))
b1 = torch.randn(100)
W2 = torch.randn((100, vocab))
b2 = torch.randn(vocab)
parameters = [C, W1, b1, W2, b2]

for p in parameters:
    p.requires_grad = True

losses = []
iters = 20
for _ in range(iters):
    # forward pass
    emb = C[X]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y)
    losses.append(loss.item())
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    for p in parameters:
        p.data += -0.1 * p.grad


plt.plot(range(iters), losses)
plt.title("Change in loss with each iteration")
plt.ylabel("loss")
plt.xlabel("iterations")
print(f"Final Loss = {losses.pop()}")

this took up way too much time. so let's do this in batches, or rather ✨*mini-batches*✨

In [ ]:
torch.randint(0, X.size(0), (32,))

In [220]:
C = torch.randn((vocab, 2))
W1 = torch.randn((6, 100))
b1 = torch.randn(100)
W2 = torch.randn((100, vocab))
b2 = torch.randn(vocab)
parameters = [C, W1, b1, W2, b2]

for p in parameters:
    p.requires_grad = True


In [221]:
iters = 10000
losses = []
for _ in range(iters):
    # mini batch
    ix = torch.randint(0, X.size(0), (200,))
    # forward pass
    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y[ix])
    # losses.append(loss.item())
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    for p in parameters:
        p.data += -0.013 * p.grad


# plt.plot(range(iters), losses)
# plt.title("Change in loss with each iteration (for the mini-batch)")
# plt.ylabel("loss")
# plt.xlabel("iterations")
# print(f"Final Loss = {losses.pop()}")


In [ ]:
emb = C[X]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y)
loss.item()

#### Well, but how do you find the optimal learning rate?

We experiment a bit to find out the range in which the rate is not cringe. Then we start from the big number of that range and slide into the lower bound with each iteration. We then finally choose the learning rate that gives the best result.

![](https://media.tenor.com/A6yz-HeOGIgAAAAM/leonardo-dicaprio-leonardo-dicaprio-meme.gif)

### Learning rate decay

We often lower the learning rate in the later stages of learning. This is called learning rate decay.

In [223]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [224]:
C = torch.randn((vocab, 2))
W1 = torch.randn((6, 100))
b1 = torch.randn(100)
W2 = torch.randn((100, vocab))
b2 = torch.randn(vocab)
parameters = [C, W1, b1, W2, b2]

for p in parameters:
    p.requires_grad = True


In [ ]:
iters = 1000
losses = []
lrs = torch.linspace(0.01, 0.15, 1000)
for i in range(iters):
    # mini batch
    ix = torch.randint(0, X.size(0), (200,))
    # forward pass
    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y[ix])
    losses.append(loss.item())
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    for p in parameters:
        p.data += -lrs[i] * p.grad


plt.plot(lrs.tolist(), losses)
plt.title("Change in loss with learing rate")
plt.ylabel("loss")
plt.xlabel("learing rate")
plt.grid()
print(f"Final Loss = {losses.pop()}")


How many parameters do we have?

In [ ]:
sum(p.nelement() for p in parameters)

### Let's get real

It shoud come at no surprise that this not how we do shit in prod. Irl, we have several splits/phases for preparing our neural net for the real world.

| ➡️ | training split | dev/validation split | test split |
|--------|----------------|----------------------|------------|
| shares | 80%            | 10%                  | 10%        |


In [ ]:
import random


def build_dataset(words):
    block_size = 8
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + "~":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]  # crop and append
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(f"{X.shape=}, {Y.shape=}")
    return X, Y


random.seed(2002)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xtest, Ytest = build_dataset(words[n2:])

In [228]:
C = torch.randn((vocab, 2))
W1 = torch.randn((6, 100))
b1 = torch.randn(100)
W2 = torch.randn((100, vocab))
b2 = torch.randn(vocab)
parameters = [C, W1, b1, W2, b2]

for p in parameters:
    p.requires_grad = True


In [ ]:
loss = None
for i in range(40000):
    # mini batch
    ix = torch.randint(0, Xtr.size(0), (200,))
    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    for p in parameters:
        p.data += -0.1 * p.grad

In [ ]:
print(loss.item())

We evaluate the performance of the neural net on our dev split. Keep in mind that these are the data that the neural net has not seen and is not trained on.

In [ ]:
# Evaluating the loss
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss.item()

Let's check what is the loss for the training set itself.

In [ ]:
# Evaluating the loss
emb = C[Xtr]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss.item()

The training set and dev set losses are not equal. This means the model is not overfitted. That implies the model is not 'memorizing' the training data.

But, since the training and dev dataset losses are roughly equal, this means that the model is underfitted. This is a good hint that our network is very tiny and we need to expand our network.


In [ ]:
C = torch.randn((vocab, 2))
W1 = torch.randn((6, 300))
b1 = torch.randn(300)
W2 = torch.randn((300, vocab))
b2 = torch.randn(vocab)
parameters = [C, W1, b1, W2, b2]

for p in parameters:
    p.requires_grad = True

sum(p.nelement() for p in parameters)

In [159]:
steps = []
losses = []

for i in range(40000):
    # mini batch
    ix = torch.randint(0, Xtr.size(0), (32,))
    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    for p in parameters:
        p.data += -0.01 * p.grad
    steps.append(i)
    losses.append(loss.item())


In [ ]:
plt.plot(steps, losses)

In [ ]:
# Evaluating the loss
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss.item()

In [ ]:
# Evaluating the loss
emb = C[Xtr]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss.item()

Visualizing the embeddings

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(C[:, 0].data.tolist(), C[:, 1].data.tolist(), s=200, color="green")
for i in range(C.size(0)):
    plt.text(
        C[i, 0].item(), C[i, 1].item(), itos[i], ha="center", va="center", color="white"
    )
plt.grid("minor")

Now that we observe we can't go much beyond our previous simple neural net, and we have already played with the weights, biases and tried expanding the number of neurons, we are begininning to suspect that the dimension of the embedding vector might be the bottleneck.

So, next we try changing that.

In [ ]:
C = torch.randn((vocab, 20))
W1 = torch.randn((60, 300)) * 0.1
b1 = torch.randn(300) * 0.01
W2 = torch.randn((300, vocab)) * 0.1
b2 = torch.randn(vocab) * 0
parameters = [C, W1, b1, W2, b2]

for p in parameters:
    p.requires_grad = True

sum(p.nelement() for p in parameters)

In [165]:
steps = []
losses = []

In [ ]:
max_steps = 50000
for i in range(max_steps):
    # mini batch
    ix = torch.randint(0, Xtr.size(0), (32,))
    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, 60) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    for p in parameters:
        p.data += -0.01 * p.grad
    steps.append(i)
    losses.append(loss.log10().item())
    # track stats
    if i % 10000 == 0:  # print every once in a while
        print(f"{i:7d}/{max_steps:7d}: {loss.item():.4f}")

In [ ]:
plt.plot(steps, losses)

In [ ]:
# Evaluating the loss
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 60) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss.item()

In [ ]:
# Evaluating the loss
emb = C[Xtr]
h = torch.tanh(emb.view(-1, 60) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss.item()

In [ ]:
# sample from the model
# g = torch.Generator().manual_seed(2002)
for _ in range(10):
    out = []
    context = [0] * block_size  # initialize with all ...
    while True:
        emb = C[torch.tensor([context])]  # (1,block_size,d)
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print("".join(itos[i] for i in out))

In [196]:
# # sample from the model
# # g = torch.Generator().manual_seed(2002)
# for _ in range(10):
#     out = []
#     context = [stoi[i] for i in "प्रदेश"]  # initialize with all ...
#     while True:
#         emb = C[torch.tensor([context])]  # (1,block_size,d)
#         h = torch.tanh(emb.view(1, -1) @ W1 + b1)
#         logits = h @ W2 + b2
#         # probs = F.softmax(logits, dim=1)
#         ix = torch.multinomial(probs, num_samples=1).item()
#         context = context[1:] + [ix]
#         out.append(ix)
#         if ix == 0:
#             break
#     print("".join(itos[i] for i in out), end=" ")
